In [11]:
import nbimporter
from pathlib import Path
import pandas as pd
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import ndcg_score

from Preprop import * # load the data_preprocess function
from Feature_En import * # load the feature_generating function

In [2]:
d_path = 'datas'
params = {'depth': 10.0, 
        'l2_leaf_reg': 5.555897323622429, 
        'learning_rate': 0.4359843989797999, 
        'loss_function': 'Logloss', 
        'max_bin': 50.0, 
        'random_strength': 0.009790073389282446
}

In [12]:
def evaluate(model, X_test, y_test, model_name, use_proba=True):
    if use_proba == True:
        y_pred = model.predict_proba(X_test)[:, 1]
    elif use_proba == False:
        y_pred =  model.predict(X_test)
    ndcg = ndcg_score(y_test.values.reshape(1, -1), y_pred.reshape(1, -1), k=20)
    print(f'NDCG@{20} на тестовых данных c использованием {model_name}: {ndcg}')

In [4]:
df = data_preprocess(d_path, use_test=True)

D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x1 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
D:\FOR STADY\python\VK_RS_relevant_tracks\venv\Lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable x4 is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(


In [5]:
df = generate_features(df)

Feature_En.ipynb:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "            dataframes_profile.loc[:, item + '_' + featur + '_std'] = dataset.groupby(item)[featur].transform('std')\n",
Feature_En.ipynb:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "        for featur in category_vars:\n",
Feature_En.ipynb:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

In [6]:
df = df.loc[:, df.columns[3:]]
train_ind = df['target'][~pd.isnull(df['target'])].index
test_ind = df['target'][pd.isnull(df['target'])].index

In [7]:
X_train = df.drop('target', axis=1).loc[train_ind]
y_train = df['target'][train_ind]
X_test = df.drop('target', axis=1).loc[test_ind].reset_index(drop=True)
y_test = df['target'][test_ind]

In [8]:
print(y_train)

print(y_test)

0          1.0
1          1.0
2          1.0
3          1.0
4          1.0
          ... 
7377413    1.0
7377414    0.0
7377415    1.0
7377416    1.0
7377417    1.0
Name: target, Length: 7377418, dtype: float32
7377418   NaN
7377419   NaN
7377420   NaN
7377421   NaN
7377422   NaN
           ..
9934203   NaN
9934204   NaN
9934205   NaN
9934206   NaN
9934207   NaN
Name: target, Length: 2556790, dtype: float32


In [13]:
model = CatBoostClassifier(n_estimators=50, random_state=0, **params)
model.fit(X_train, y_train)

evaluate(model, X_test, y_test, 'CatBoostClassifier', use_proba=False)
evaluate(model, X_test, y_test, 'CatBoostClassifier', use_proba=True)

0:	learn: 0.6600035	total: 803ms	remaining: 39.4s
1:	learn: 0.6438053	total: 1.69s	remaining: 40.6s
2:	learn: 0.6353082	total: 2.55s	remaining: 40s
3:	learn: 0.6295977	total: 3.3s	remaining: 38s
4:	learn: 0.6263402	total: 4.07s	remaining: 36.6s
5:	learn: 0.6237199	total: 4.75s	remaining: 34.8s
6:	learn: 0.6210758	total: 5.43s	remaining: 33.3s
7:	learn: 0.6192929	total: 6.21s	remaining: 32.6s
8:	learn: 0.6180612	total: 6.88s	remaining: 31.4s
9:	learn: 0.6162802	total: 7.6s	remaining: 30.4s
10:	learn: 0.6149208	total: 8.38s	remaining: 29.7s
11:	learn: 0.6137866	total: 9.14s	remaining: 28.9s
12:	learn: 0.6130562	total: 9.89s	remaining: 28.2s
13:	learn: 0.6121478	total: 10.7s	remaining: 27.4s
14:	learn: 0.6115452	total: 11.4s	remaining: 26.6s
15:	learn: 0.6109270	total: 12s	remaining: 25.6s
16:	learn: 0.6104388	total: 12.8s	remaining: 24.8s
17:	learn: 0.6098069	total: 13.6s	remaining: 24.2s
18:	learn: 0.6092572	total: 14.4s	remaining: 23.5s
19:	learn: 0.6085981	total: 15.2s	remaining: 22.8

ValueError: Input contains NaN.